In [1]:
# IN THIS CELL WE INITIALIZE THE CODE AND COMPUTE THE TAYLOR SERIES OF P, Q
# UP TO ORDER K. THIS COMPUTATION IS SYMBOLIC, AND WE STORE THE TAYLOR COEFFICIENTS
# AS POLYNOMIALS OF lam 
#===================================================================================

# lam is a symbolic variable for the eigenvalue
# L is the angular mode for the Dirichlet eigenfunction which is always 8 in our paper
# K is the number of computed Taylor series coefficients
lam = var('lam')
L = 8
K = 100

# We set the precision for our real ball field
# We will mainly work with RR which defines intervals via balls (center & radius)
# We use RIF when we want to define an interval via its endpoints
prec = 200
RR = RealBallField(prec)
RIF = RealIntervalField(prec)

gamma = RR(3)/RR(2)

# We implement the Taylor series coefficients from equations (2.5)-(2.6)
def Taylor_Coeffs(K_max, ell):
    pp = list(var('p_%d' % i) for i in (0..K_max))
    qq = list(var('q_%d' % i) for i in (0..K_max))

    # Initial conditions for the recurrences
    # The terms 0*lam ensure Sage stores them as polynomials in lambda
    # For the index k = 0 we just store in pp[0] and qq[0] the value of the Taylor coefficient
    # without the k(k+ell) division (which would be singular for k = 0)
    pp[0] = 1+0*lam
    pp[1] = -lam
    pp[2] = lam * (2 + lam / (1 + ell)) 
    
    qq[0] = 0*lam
    qq[1] = -1+0*lam
    qq[2] = 2 + 2 * lam / (1+ell)

    # Recurrence for p_k 
    # p_{k+1} = -(2 + lam/(k*(k+ell)))*p_k - p_{k-1}
    # Recurrence for q_k
    # q_{k+1} = -(2 + lam/(k*(k+ell)))*q_k - (1/(k*(k+ell)))*p_k - q_{k-1}
    for k in (2..K_max-1):
        factor = (2 + lam / (k * (k + ell))).expand()
        pp[k+1] = (-factor * pp[k] - pp[k-1]).expand()
        qq[k+1] = (-factor * qq[k] - (1 / (k * (k + ell))) * pp[k] - qq[k-1]).expand()
        
    return pp, qq

# We store the radial coefficients with neu (since they are used for Neumann)
# We store the coefficients with mode L with dir (since they are used for Dirichlet)
pp_neu, qq_neu = Taylor_Coeffs(K+2, 0)
pp_dir, qq_dir = Taylor_Coeffs(K+2, L)

In [2]:
# IN THIS CELL WE IMPLEMENT A INTERVAL ARITHMETIC VERSION OF THE LEGENDRE FUNCTIONS
# AS WELL AS THEIR DERIVATIVES WITH RESPECT TO RHO OR LAMBDA
#==============================================================================

def my_legendre(lam_V, rho_V, ell, char):
    """
    Rigorously computes the associated Legendre polynomial P or its indicial
    derivative Q (and their first derivatives) using a Taylor series in rho
    and a rigorous tail bound.

    Args:
        lam_V (RealBall): The eigenvalue lambda.
        rho_V (RealBall): The coordinate rho = tan(theta/2).
        ell (int): The angular mode (0 for Neumann, 8 for Dirichlet).
        char (str): Specifies the function to compute:
                    'P' for P_{alpha, l}
                    'P_prime' for d/d(rho) P_{alpha, l}
                    'Q' for d/d(lam) P_{alpha, l}
                    'Q_prime' for d/d(rho) d/d(lam) P_{alpha, l}

    Returns:
        RealBall: The computed value as an interval.
    """

    # We select the appropriate Taylor series depending on ell (0 or 8) and 
    # having a P-type series or a Q-type series
    if ell == 8:
        if char in ['P', 'P_prime']:
            poly_coeffs_symbolic = pp_dir
        elif char in ['Q', 'Q_prime']:
            poly_coeffs_symbolic = qq_dir
        else:
            raise ValueError("Invalid character for ell=8. Use 'P', 'P_prime', 'Q', or 'Q_prime'.")
    elif ell == 0:
        if char in ['P', 'P_prime']:
            poly_coeffs_symbolic = pp_neu
        elif char in ['Q', 'Q_prime']:
            poly_coeffs_symbolic = qq_neu
        else:
            raise ValueError("Invalid character for ell=0. Use 'P', 'P_prime', 'Q', or 'Q_prime'.")
    else:
        raise ValueError("Error: We only precomputed the series for ell = 0, 8.")

    # ======== We substitute lam by lam_V on poly_coeffs_symbolic and obtain intervals containing the Taylor coefficients =====
    # The fundamental property is that the interval coefs[i] contains poly_coeffs_symbolic[i] for any value of lamb in the interval lam_V 
    subs = {lam: lam_V}
    coefs = [RR(0)] * (K + 2)
    for i in range(K + 2):
        coefs[i] = RR(poly_coeffs_symbolic[i](subs))

    # === We compute the main sum (the truncated series) ===
    # This is done for the function value or its derivative based on 'char'.
    if char in ['P', 'Q']:
        # We initialize with pp[0] = 1 if P or qq[0] = 0 if Q
        series_sum = RR(0)
        if char == 'P':
            series_sum = RR(1)
        
        # We sum the contributions up to K
        for k in range(1, K):
            series_sum += coefs[k] / (RR(k) * (RR(k) + RR(ell))) * rho_V**(2*k)
        
        # Multiply by the outer rho^ell factor
        main_sum = rho_V**ell * series_sum

    elif char in ['P_prime', 'Q_prime']:
        # In the case of derivatives we compute series for both the function and its derivative
        # For exmaple, for char = P_prime we compute the main term for both P (series_sum_val) and P_prime (series_deriv_sum)
        # since the series P will also appear when the derivative hits the rho^ell term

        # We initialize
        series_sum_val = RR(0)
        series_deriv_sum = RR(0)
        if char == 'P_prime':
            series_sum_val = RR(1)
            
        # We compute the series
        for k in range(1, K):
            term = coefs[k] / (RR(k) * (RR(k) + RR(ell))) * rho_V**(2*k)
            series_sum_val += term
            
            deriv_term = coefs[k] / (RR(k) * (RR(k) + RR(ell))) * (2*k) * rho_V**(2*k-1)
            series_deriv_sum += deriv_term
        
        # Combine parts using the product rule.
        main_sum = ell * rho_V**(ell-1) * series_sum_val + rho_V**ell * series_deriv_sum

    else:
        raise ValueError(f"char argument must be P, Q, P_prime or Q_prime")
        
    #print("Main sum = ", main_sum)

    
    # === Now, we compute the rigorous error bound for the tail ===
    # === We start by checking the conditions on our Lemma 2.4

    # ==== We verify the condition from the Lemma for the induction to hold:
    # (gamma - 1)^2 / gamma >= 2 * lam / ((K+2)*(K+2+ell))

    lhs_check = (gamma - RR(1))**2 / gamma
    rhs_check = 2 * abs(lam_V) / (RR(K+2) * (RR(K+2) + RR(ell)))
    if lhs_check.upper() < rhs_check.lower():
            raise ValueError(f"Gamma condition failed: {lhs_check} is strictly less than {rhs_check}")
    #else:
    #        print("Gamma condition verified.")

    # ======= We determine the bounding constant C from Lemma 2.4.
    # C must bound |p_{K+1}+p_K|, |q_{K+1}+q_K|, etc.
    if ell == 0:
        bound1 = abs(RR(pp_neu[K+1](subs)) + RR(pp_neu[K](subs))) / lam_V
        bound2 = abs(RR(qq_neu[K+1](subs)) + RR(qq_neu[K](subs)))
        bound3 = abs(RR(pp_neu[K+1](subs))) * (gamma - RR(1)) / lam_V
        bound4 = abs(RR(qq_neu[K+1](subs))) * (gamma - RR(1))

    elif ell == 8:
        bound1 = abs(RR(pp_dir[K+1](subs)) + RR(pp_dir[K](subs))) / lam_V
        bound2 = abs(RR(qq_dir[K+1](subs)) + RR(qq_dir[K](subs)))
        bound3 = abs(RR(pp_dir[K+1](subs))) * (gamma - RR(1)) / lam_V
        bound4 = abs(RR(qq_dir[K+1](subs))) * (gamma - RR(1))
    else: 
        raise ValueError(f"We only have implementations for ell = 0, 8")

    C = max(bound1, bound2, bound3, bound4)
    #print("C has value ", C, " and type ", type(C))

    # ====== We verify the radius of convergence condition rho^2 gamma < 1
    rho_sq_gamma = abs(rho_V**2 * gamma)
    if rho_sq_gamma.upper() >= 1:
        raise ValueError(f"The tail does not converge for those values of gamma, rho ")
    #else: 
    #    print("rho_V contained in the ball of convergence and rho/ROC = ", sqrt(rho_sq_gamma))


    # ======== We compute the tail error using the formulas from Lemma 2.4
    # The common factor to all tail bounds is C rho^(2K+ell+1)gamma / ((gamma-1)(1-rho^2gamma) K)
    
    main_factor = C * rho_V^(RR(2*K) + RR(ell) + RR(1)) * gamma / ((gamma - RR(1)) * (RR(1)-rho_sq_gamma) * RR(K))

    # Now, we separate on each case for the tail bound
    if char == 'P':
        tail_error = main_factor * rho_V * lam_V / RR(K+ell)
    elif char == 'P_prime':
        tail_error = main_factor * RR(2) * lam_V
    elif char == 'Q':
        tail_error = main_factor * rho_V / RR(K+ell)
    elif char == 'Q_prime':
        tail_error = main_factor * RR(2)
    else:
        raise ValueError(f"char argument must be P, Q, P_prime or Q_prime")

    # ====== Finally, since we have the main_sum and the bound for the tail contribution, we simply add the tail as an error
    #print("Sum: ", main_sum)
    #print("Tail error: ", tail_error)
    error_radius = tail_error.upper()

    error_interval = RR(RIF(-error_radius, error_radius))
    final_result = main_sum + error_interval
    return final_result

In [3]:
# IN THIS CELL WE PROVE THE FULL LEMMA 2.5 EXCEPT FOR THE FIRST ITEM OF (AD)
# WHICH IS DEFERRED TO THE NEXT CELLS

# We define the endpoints of our box. They are intervals which are ensured to
# contain the exact values of rho_l, rho_u, lam_l, lam_u
rho_l = RR(594723480694931) / RR(1e15)
rho_u = RR(594723480694970) / RR(1e15)
lam_l = RR(154191574494505) / RR(1e12)
lam_u = RR(154191574494520) / RR(1e12)

# Now we define the intervals defining our box
rho_I = RR(RIF(rho_l.lower(), rho_u.upper()))
lam_I = RR(RIF(lam_l.lower(), lam_u.upper()))

# We recover nu and a via nu = -1/2 + sqrt(1 + lam)/2
# and a = (1-rho^2)/(1+rho^2)
a_I = (RR(1) - rho_I**2) / (RR(1) + rho_I**2)
nu_I = -RR(1)/RR(2) + sqrt(RR(1)/RR(4) + lam_I)
print("Interval for a = [", a_I.lower(), ", ", a_I.upper(), "]")
print("\nInterval for nu = [", nu_I.lower(), ", ", nu_I.upper(), "]")


#==== Now, we proceed with the proof of the Lemma 2.5
#==== We start with the proof of CAP-B

print("\n===================")
print("Proof of (CAP-B)")
print("===================\n")
print("First line of conditions ")
print("P_{lam_l, 8}(rho_u) = ", my_legendre(lam_l, rho_u, 8, 'P'))
print("P_{lam_u, 8}(rho_l) = ", my_legendre(lam_u, rho_l, 8, 'P'))
print("(d/drho) P_{lam, 8}(rho) = ", my_legendre(lam_I, rho_I, 8, 'P_prime'))
print("--------------------------")

print("\n Second line of conditions ")
print("P_{lam_l, 0}'(rho_u) = ", my_legendre(lam_l, rho_u, 0, 'P_prime'))
print("P_{lam_u, 0}'(rho_l) = ", my_legendre(lam_u, rho_l, 0, 'P_prime'))
print("(d/dlambda) P_{lam, 0}'(rho) = ", my_legendre(lam_I, rho_I, 0, 'Q_prime'))


#========== Now we proceed with the proof of the second part of (AN)

print("\n===========================")
print("Proof of second part of (CAP-AD)")
print("===========================\n")

lam_aux = RR(1541914)/RR(10000)
my_interval = RR(RIF(lam_aux.lower(), lam_u.upper()))
result = my_legendre(my_interval, rho_I, 8, 'Q')

# Sage sometimes print bigger intervals than the ones computed, in order
# to get shorter expressions (still, always guaranteeing that the printed interval contains
# the interval being computed, so that the printed result is still rigorous).
# To avoid that, for this computation, we explicitly print the lower and upper bound:
print("Q_{lam^aux}^8 (rho^I) = [", result.lower(), ", ", result.upper(), "]")


#======= We proceed with the proof of CAP-LN

print("\n===========================")
print("Proof of second part of (CAP-LN)")
print("===========================\n")

print("P_{12}^0'(rho)= ", my_legendre(RR(12), rho_I, 0, 'P_prime'))
print("P_{13}^0'(rho)= ", my_legendre(RR(13), rho_I, 0, 'P_prime'))
print("P_{42}^0'(rho)= ", my_legendre(RR(42), rho_I, 0, 'P_prime'))
print("P_{43}^0'(rho)= ", my_legendre(RR(43), rho_I, 0, 'P_prime'))
print("P_{89}^0'(rho)= ", my_legendre(RR(89), rho_I, 0, 'P_prime'))
print("P_{90}^0'(rho)= ", my_legendre(RR(90), rho_I, 0, 'P_prime'))


#========= Proof of transversality

print("\n===================")
print("Proof of (CAP-T)")
print("===================\n")

first_term_Q = my_legendre(lam_I, rho_I, 8, 'Q')
second_term_Q = my_legendre(lam_I, rho_I, 0, 'Q_prime')
first_term_P = RR(4) * lam_I / (RR(1)+rho_I**2)**2 * my_legendre(lam_I, rho_I, 0, 'P')
second_term_P = my_legendre(lam_I, rho_I, 8, 'P_prime')

transversality_term = first_term_Q * first_term_P + second_term_Q * second_term_P
print("Transversality term = ", transversality_term)
print("Q_lambda^8 (rho) = ", first_term_Q)
print("Q_lambda^0' (rho) = ", second_term_Q)


# ===== Proof of non-resonance
print("\n===================")
print("Proof of (CAP-NR)")
print("===================\n")

result = my_legendre(lam_I, rho_I, 0, 'P')
print("P_lambda^0 (rho) = ", result)

Interval for a = [ 0.47743656824152983791576680455606002118074115969648119062185 ,  0.47743656824158046745947488016738498306512639146603760309518 ]

Interval for nu = [ 11.927452453922525463780315250345021027555958426091820001602 ,  11.927452453923128975605655018379924747762288461672142148018 ]

Proof of (CAP-B)

First line of conditions 
P_{lam_l, 8}(rho_u) =  [1.17476454929e-18 +/- 3.63e-30]
P_{lam_u, 8}(rho_l) =  [-9.1354964023e-19 +/- 2.30e-30]
(d/drho) P_{lam, 8}(rho) =  [-0.000492206 +/- 5.42e-10]
--------------------------

 Second line of conditions 
P_{lam_l, 0}'(rho_u) =  [-1.2902088e-13 +/- 4.71e-21]
P_{lam_u, 0}'(rho_l) =  [1.6833517e-13 +/- 5.27e-21]
(d/dlambda) P_{lam, 0}'(rho) =  [-0.19131 +/- 6.15e-6]

Proof of second part of (CAP-AD)

Q_{lam^aux}^8 (rho^I) = [ -2.5900295281733976481515625376242406494845831604764116061748e-6 ,  -2.4790164276040532067750486321628166510958316047641160617472e-7 ]

Proof of second part of (CAP-LN)

P_{12}^0'(rho)=  [-0.2720875484 +/- 6.69e-

In [4]:
# Here we implement our branch and bound algorithm in order to prove the first part of (AD) in Lemma 2.5

# Check_interval tries to check if the function is != 0 on the interval left_V.lower(), right_V.upper()
# by considering mid_V to be the midpoint, checking the value of the function there, and using
# intermediate value theorem with the derivative being computed on the full interval containing left_V, right_V

# We refer to the explanation on the proof of Lemma 2.5 for the specific application 
def check_interval_empty(left_V, right_V, rho_V, ell, char):
    # We create an interval containing both left_V and right_V
    int_V = RR(RIF(left_V.lower(), right_V.upper()))
    
    # We compute d/dlambda(legendre) on the full interval
    # Recall from the proof of Lemma 2.5 that we call that interval beta
    if char == 'P':
        beta = my_legendre(int_V, rho_V, ell, 'Q')
    elif char == 'P_prime':
        beta = my_legendre(int_V, rho_V, ell, 'Q_prime')
    else:
        raise ValueError(f"char argument must be P or Q to have an available lambda derivative")
    
    # We compute the midpoint and radius. Recall from the proof of Lemma 2.5 that
    # alpha contains the Legendre function evaluated at lambda = mid_V and rho = rho_V
    mid_V = (left_V + right_V)/RR(2)
    rad_V = (right_V - left_V)/RR(2)
    alpha = my_legendre(mid_V, rho_V, ell, char)

    # We check the interval value theorem condition to verify the Legendre function does not vanish
    # We refer to the proof of Lemma 2.5 for the derivation of that condition
    rad_ball = rad_V.union(-rad_V)
    res_V = alpha + rad_ball * beta
    return (res_V != 0)

# Here, we implement our branch and bound algorithm
def recursive_check(left_V, right_V, rho_V, ell, char):
    print("I am looking at [", left_V.n(), ", ", right_V.n(), "]")

    # If the interval is of size < 1e-6 we stop and the check fails
    if right_V.lower() - left_V.upper() < 1e-6:
        raise ValueError(f"Tolerance reached at [", left_V, ", ", right_V, "]")

    # We try to check the whole interval [left_V, right_V]
    # If we succeed, we return true
    # If we do not succeed, we split the interval in two halves and try again each half
    bool0 = check_interval_empty(left_V, right_V, rho_V, ell, char)
    if bool0:
        return True
    else:
        mid_V = (left_V + right_V)/2
        bool1 = recursive_check(left_V, mid_V, rho_V, ell, char)
        bool2 = recursive_check(mid_V, right_V, rho_V, ell, char)
    return (bool1 & bool2)
    

In [ ]:
# We execute the branch and bound to check that P_{lambda}^8 (rho) != 0 for all lambda between 2 and lam_aux
 
time1 = cputime()
recursive_check(RR(2), lam_aux, rho_I, 8, 'P')
time2 = cputime()
print("Total time for the branch and bound: ", time2-time1)